# Debug

In [ ]:
### RuntimeError: cuda runtime error (59) : device-side assert triggered ###

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Memory

In [ ]:
import psutil

def memory_func(name):
    print(f"{name}: {psutil.virtual_memory().percent}")

def memory_usage(fn, *args, **kwargs):
    before = psutil.virtual_memory().used
    res = fn(*args, **kwargs)
    after = psutil.virtual_memory().used
    mem = after-before
    return res, round(mem / math.pow(1024,2), 2)

# Images

In [ ]:
def show_img(im, figsize=None, ax=None, alpha=None, title=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(image2np(im.data), alpha=alpha)
    if title: ax.set_title(title)
    return ax

# Model Telemetry

In [ ]:
from fastai.callbacks.hooks import *

In [ ]:
class FullStats(HookCallback):
    def on_train_begin(self, **kwargs):
        self.modules = [m for m in flatten_model(self.learn.model) if hasattr(m, 'weight')]
        self.g_hooks = Hooks(self.modules, self.g_hook, is_forward=False)
        self.a_hooks = Hooks(self.modules, self.a_hook)
        self.grads,self.acts = [],[]

    def g_hook(self, m:nn.Module, i:Tensors, o:Tensors)->Tuple[Rank0Tensor,Rank0Tensor]:
        oo = next(o)
        return oo.mean().item(),oo.std().item()
    
    def a_hook(self, m:nn.Module, i:Tensors, o:Tensors)->Tuple[Rank0Tensor,Rank0Tensor]:
        return o.mean().item(),o.std().item()

    def on_batch_end(self, train, **kwargs):
        if train:
            self.acts.append(self.a_hooks.stored)
            self.grads.append(self.g_hooks.stored)
            
    def on_train_end(self, **kwargs):
        self.a_hooks.remove()
        self.g_hooks.remove()
        self.acts = tensor(self.acts).permute(2,1,0)
        self.grads = tensor(self.grads).permute(2,1,0)

In [ ]:
learn.fit(1, 1e-5, callbacks=[FullStats(learn)])#, StopAfterNBatches(n_batches=2)])

In [ ]:
acts,grads = learn.full_stats.acts, learn.full_stats.grads
acts.shape,grads.shape

In [ ]:
avg_act_stds_by_layer = acts[1,:].mean(-1)
avg_grad_stds_by_layer = grads[1,:].mean(-1)

plt.plot(avg_act_stds_by_layer)

plt.plot(avg_grad_stds_by_layer)

for (i,mod),a,g in zip(enumerate(names), avg_act_stds_by_layer, avg_grad_stds_by_layer):
    mod_name = str(mod).split('(')[0]
    print(f"{str(i).ljust(3)} {mod_name.ljust(60)} \
            {str(round(a.item(),5)).ljust(6)} {str(round(g.item(),5)).ljust(6)}")

# Last batch activations by layer

for (i,mod),m,s in zip(enumerate(names), acts[0,:,-1], acts[1,:,-1]):
    mod_name = str(mod).split('(')[0]
    print(f"{str(i).ljust(3)} {mod_name.ljust(50)} \
            {str(round(m.item(),5)).ljust(6)}  {str(round(s.item(),5)).ljust(6)}")